# Bibliotecas

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
from obterDados import obterSimbolo

# Obtendo dados

In [ ]:
rawDataM5 = obterSimbolo('WDO$')
rawDataM5

# Tratamento de Dados

### Gerando colunas com preços passados e futuros

In [ ]:
hist = rawDataM5.copy().drop(columns=['spread'])
colunas0 = ['open', 'high', 'low', 'close', 'tick_volume', 'real_volume']
for i in range(1, 1000):
    for coluna in colunas0:
        hist[coluna+str(i)] = hist[coluna].shift(i)
colunasY = []
for i in range(1, 25):
    for coluna in colunas0:
        nomeCol = f'{coluna}-{str(i)}'
        hist[nomeCol] = hist[coluna].shift(-i)
        colunasY.append(nomeCol)

hist = hist.dropna()
hist

# Modelo de Classificação

### Normalização

In [ ]:
# maxVs = hist.drop(columns=colunasY).max(axis='columns')
# minVs = hist.drop(columns=colunasY).min(axis='columns')
# histNorm = hist.subtract(minVs, axis=0).divide(maxVs - minVs, axis=0)
# histNorm

### Achar quando comprar, vender ou fazer nada

In [ ]:
hist['action'] = 0
ganhoEmPontos = 30
actions = hist['action'].to_numpy()
fechamentos = hist['close'].to_numpy()
for i in range(len(fechamentos)):
    for j in range(1, 25):
        nameCol = f'high-{str(j)}'
        if hist[nameCol].to_numpy()[i] >= fechamentos[i] + ganhoEmPontos:
            actions[i] = 1
            break
        nameCol = f'low-{str(j)}'
        if hist[nameCol].to_numpy()[i] <= fechamentos[i] - ganhoEmPontos:
            actions[i] = 2
            break
hist['action'] = actions.tolist()
colunasY.append('action')
colunasY

### Separar dados de treino e teste

In [ ]:
xClass = hist.drop(columns=colunasY)
yClass = hist['action']
splitDays = 108 * 150
x_train_class = xClass.iloc[:-splitDays]
x_test_class = xClass.iloc[-splitDays:]
y_train_class = yClass.iloc[:-splitDays]
y_test_class = yClass.iloc[-splitDays:]

### Lazy Classifier

In [ ]:
from lazypredict.Supervised import LazyClassifier

lazyReg = LazyClassifier()
lazyReg.fit(x_train_class, x_test_class, y_train_class, y_test_class)

### Criando um dos modelos encontrados

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

# modelo = MLPClassifier(verbose=True)
modelo = AdaBoostClassifier()
modelo.fit(x_train_class, y_train_class)


### Salvar Modelo de Classificação

In [ ]:
import pickle

pickle.dump(modelo, open('modeloClassificador.pickle', 'wb'))

# Pontuação de treino e teste

In [ ]:
from sklearn.metrics import classification_report
from pickle import load

modelo = load(open('modeloClassificador.pickle', 'rb'))
print('Treino: ', classification_report(y_train_class, modelo.predict(x_train_class)))
print('Teste: ', classification_report(y_test_class, modelo.predict(x_test_class)))